In [123]:
from doubt import Boot
from doubt.datasets import (Airfoil, Blog, Concrete, CPU, 
                            FacebookComments, FishBioconcentration,
                            FishToxicity, ForestFire, NewTaipeiHousing,
                            PowerPlant, Protein, Servo,
                            SpaceShuttle)
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import (LinearRegression, PoissonRegressor, 
                                  GammaRegressor, HuberRegressor)
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from tqdm.auto import tqdm
from collections import defaultdict

## Setting up

In [229]:
def evaluate_nasa(model, X_tr, X_te, y_tr, y_te, uncertainty=0.05):
    n_boots = int(np.sqrt(len(X_tr)))
    
    # Calculate training residuals
    model.fit(X_tr, y_tr)
    tr_preds = model.predict(X_tr)
    te_preds = model.predict(X_te)
    tr_residuals = y_tr - tr_preds
    
    n_train = X_tr.shape[0]
    n_test = X_te.shape[0]

    # Initialise random number generator
    rng = np.random.default_rng(4242)

    # Compute the model variances
    bootstrap_preds = np.empty((n_boots, n_test))
    for boot_idx in range(n_boots):
        train_idxs = rng.choice(range(n_train), size=n_train, replace=True)
        X_btr = X_tr[train_idxs, :]
        y_btr = y_tr[train_idxs]

        model.fit(X_btr, y_btr)
        
        bootstrap_pred = model.predict(X_te)
        bootstrap_preds[boot_idx] = bootstrap_pred

    # Centre the bootstrapped predictions across the bootstrap dimension
    bootstrap_preds = np.mean(bootstrap_preds, axis=0) - bootstrap_preds

    # Add up the bootstrap predictions and the hybrid train/val residuals
    C = np.array([m + o for m in bootstrap_preds for o in tr_residuals])

    # Calculate the intervals
    intervals = np.expand_dims(te_preds, -1) + np.transpose(np.quantile(C, q=[uncertainty/2, 1-uncertainty/2], axis=0))
    
    coverage = np.mean((y_te > intervals[:, 0]) & (y_te < intervals[:, 1]))
    mean_width = np.mean(intervals[:, 1] - intervals[:, 0])
    return coverage, mean_width

In [230]:
def evaluate_doubt(model, X_tr, X_te, y_tr, y_te, uncertainty=0.05):
    n_boots = int(np.sqrt(len(X_tr)))
    
    bmodel = Boot(model, random_seed=4242)
    bmodel.fit(X_tr, y_tr, n_boots=n_boots)
    preds, intervals = bmodel.predict(X_te, uncertainty=uncertainty, n_boots=n_boots)
    
    coverage = np.mean((y_te > intervals[:, 0]) & (y_te < intervals[:, 1]))
    mean_width = np.mean(intervals[:, 1] - intervals[:, 0])
    return coverage, mean_width

In [231]:
datasets = []
scaler = StandardScaler()

# Add Doubt datasets
dataset_classes = [Airfoil, Concrete, FishToxicity, NewTaipeiHousing, ForestFire]
for dataset_class in dataset_classes:
    X_tr, X_te, y_tr, y_te = dataset_class().split(test_size=0.1, random_seed=4242)
    X_tr = scaler.fit_transform(X_tr)
    X_te = scaler.transform(X_te)
    datasets.append((dataset_class.__name__, X_tr, X_te, y_tr, y_te))
    
len(datasets)

5

## Linear Regression

In [232]:
model = LinearRegression()

In [233]:
nasa_coverages = list()
nasa_mean_widths = list()
doubt_coverages = list()
doubt_mean_widths = list()

data_dict = defaultdict(list)
for dataset in datasets:
    for uncertainty in [0.01, 0.05, 0.1]:
        nasa_coverage, nasa_mean_width = evaluate_nasa(model, *dataset[1:], uncertainty=uncertainty)
        doubt_coverage, doubt_mean_width = evaluate_doubt(model, *dataset[1:], uncertainty=uncertainty)
        data_dict['dataset'].append(dataset[0])
        data_dict['uncertainty'].append(uncertainty)
        data_dict['nasa_coverage'].append(100 * nasa_coverage)
        data_dict['doubt_coverage'].append(100 * doubt_coverage)
        data_dict['nasa_mean_width'].append(nasa_mean_width)
        data_dict['doubt_mean_width'].append(doubt_mean_width)
    
df = pd.DataFrame(data_dict).set_index(['dataset', 'uncertainty'])
df

nasa_coverage  doubt_coverage  nasa_mean_width  \
dataset          uncertainty                                                   
Airfoil          0.01             99.280576       99.280576        28.293689   
                 0.05             92.086331       92.805755        19.489191   
                 0.10             87.769784       87.050360        15.365684   
Concrete         0.01             97.196262      100.000000        52.676151   
                 0.05             90.654206       91.588785        41.318408   
                 0.10             86.915888       86.915888        34.658419   
FishToxicity     0.01             97.849462       98.924731         5.814432   
                 0.05             91.397849       92.473118         3.928487   
                 0.10             90.322581       91.397849         3.031887   
NewTaipeiHousing 0.01             97.560976       97.560976        50.892050   
                 0.05             97.560976       95.121951        34.977061   
                 0.10             90.243902       95.121951        23.787838   
ForestFire       0.01            100.000000       98.076923       304.860939   
                 0.05             98.076923       98.076923       104.931275   
                 0.10             94.230769       98.076923        71.186709   

                              doubt_mean_width  
dataset          uncertainty                    
Airfoil          0.01                30.396459  
                 0.05                19.479813  
                 0.10                15.404273  
Concrete         0.01                56.064162  
                 0.05                42.282408  
                 0.10                35.246152  
FishToxicity     0.01                 7.529623  
                 0.05                 3.834488  
                 0.10                 3.033560  
NewTaipeiHousing 0.01                63.776944  
                 0.05                32.565166  
                 0.10                24.182167  
ForestFire       0.01               224.905686  
                 0.05                99.748985  
                 0.10                73.846014

## Decision Tree

In [234]:
model = DecisionTreeRegressor()

In [235]:
nasa_coverages = list()
nasa_mean_widths = list()
doubt_coverages = list()
doubt_mean_widths = list()

data_dict = defaultdict(list)
for dataset in datasets:
    for uncertainty in [0.01, 0.05, 0.1]:
        nasa_coverage, nasa_mean_width = evaluate_nasa(model, *dataset[1:], uncertainty=uncertainty)
        doubt_coverage, doubt_mean_width = evaluate_doubt(model, *dataset[1:], uncertainty=uncertainty)
        data_dict['dataset'].append(dataset[0])
        data_dict['uncertainty'].append(uncertainty)
        data_dict['nasa_coverage'].append(100 * nasa_coverage)
        data_dict['doubt_coverage'].append(100 * doubt_coverage)
        data_dict['nasa_mean_width'].append(nasa_mean_width)
        data_dict['doubt_mean_width'].append(doubt_mean_width)
    
df = pd.DataFrame(data_dict).set_index(['dataset', 'uncertainty'])
df

nasa_coverage  doubt_coverage  nasa_mean_width  \
dataset          uncertainty                                                   
Airfoil          0.01             92.086331       99.280576         9.267137   
                 0.05             92.086331       98.561151         9.062791   
                 0.10             87.050360       97.841727         7.098770   
Concrete         0.01             88.785047      100.000000        20.911108   
                 0.05             81.308411       97.196262        20.788676   
                 0.10             72.897196       91.588785        14.946832   
FishToxicity     0.01             91.397849      100.000000         3.632771   
                 0.05             79.569892       93.548387         3.117494   
                 0.10             70.967742       90.322581         2.470283   
NewTaipeiHousing 0.01             80.487805      100.000000        20.671988   
                 0.05             63.414634       95.121951        19.773171   
                 0.10             68.292683       92.682927        19.629268   
ForestFire       0.01             84.615385       98.076923       171.004125   
                 0.05             75.000000       98.076923       164.178077   
                 0.10             78.846154       90.384615       120.015981   

                              doubt_mean_width  
dataset          uncertainty                    
Airfoil          0.01                22.037481  
                 0.05                12.057218  
                 0.10                 9.696030  
Concrete         0.01                64.683975  
                 0.05                28.645234  
                 0.10                22.514410  
FishToxicity     0.01                 8.414255  
                 0.05                 4.715847  
                 0.10                 3.800851  
NewTaipeiHousing 0.01                71.722853  
                 0.05                32.951871  
                 0.10                25.970193  
ForestFire       0.01              1150.727066  
                 0.05               330.583178  
                 0.10               183.387744